In [1]:
import sympy as sp
from sympy.physics.quantum import TensorProduct, Dagger
from sympy.physics.quantum.trace import Tr
import random
import numpy as np

def half_wave_plate_sympy(theta):
    c = sp.cos(2 * theta)
    s = sp.sin(2 * theta)
    return sp.Matrix([[c, s], [s, -c]])

# Define theta as a symbolic variable and create operator
theta_1, theta_2, Lambda = sp.symbols('theta_1 theta_2 lambda', real=True)
HWP_operator_1 = half_wave_plate_sympy(theta_1)
HWP_operator_2 = half_wave_plate_sympy(theta_2)
LP_operator = sp.Matrix([[1, 0], [0, 0]])

#polariser_operator = TensorProduct(sp.Matrix([[1, 0],[0, 0]]), sp.eye(2))

# Define horizontal and vertical polarization states
H = sp.Matrix([1, 0])
V = sp.Matrix([0, 1])

# Bell state (vector)
Phi_plus = (TensorProduct(H, H) + TensorProduct(V, V)) / sp.sqrt(2)
Phi_minus = (TensorProduct(H, H) - TensorProduct(V, V)) / sp.sqrt(2)
Psi_plus = (TensorProduct(H, V) + TensorProduct(V, H)) / sp.sqrt(2)
Psi_minus = (TensorProduct(H, V) - TensorProduct(V, H)) / sp.sqrt(2)

Phi_mixed = (TensorProduct(H, H) + sp.exp(Lambda *sp.I) * TensorProduct(V, V)) / sp.sqrt(2)


# Apply HWP to the first qubit of the entangled state
HWP_total_operator = TensorProduct(HWP_operator_1, HWP_operator_2)
LP_total_operator = TensorProduct(LP_operator, sp.eye(2))
after_HWP_state = HWP_total_operator * Phi_plus
sp.simplify(after_HWP_state).subs({theta_1: 0, theta_2: 0})


Matrix([
[sqrt(2)/2],
[        0],
[        0],
[sqrt(2)/2]])

In [2]:
# Beam cube implementation:

# Define the projection operator for vertically & horizontally polarized light
P_V = V * Dagger(V)  # |V><V| projection operator
P_H = H * Dagger(H)  # |H><H| projection operator

# Projects into each combination of 2 output ports from either beam line
VH_operator = TensorProduct(P_V, P_H)
VV_operator = TensorProduct(P_V, P_V)
HH_operator = TensorProduct(P_H, P_H)
HV_operator = TensorProduct(P_H, P_V)

after_HWP_state.subs({theta_1: 0, theta_2: 0})

# Probability of getting coincidence in HV port
(after_HWP_state.T * HV_operator * after_HWP_state).subs({theta_1:sp.pi/4, theta_2:0})[0]

1/2

### Density Matrix Formalism

In [3]:
rho_phi_plus = TensorProduct(Phi_plus, Dagger(Phi_plus))
rho_phi_minus = TensorProduct(Phi_minus, Dagger(Phi_minus))
rho_psi_plus = TensorProduct(Psi_plus, Dagger(Psi_plus))
rho_psi_minus = TensorProduct(Psi_minus, Dagger(Psi_minus))

rho_phi_mixed = TensorProduct(Phi_mixed, Dagger(Phi_mixed))

mixed_state = 0.5*TensorProduct(TensorProduct(H, H), Dagger(TensorProduct(H,H))) + 0.5*TensorProduct(TensorProduct(V, V), Dagger(TensorProduct(V,V)))

rho_after_HWP = sp.simplify(HWP_total_operator * rho_phi_plus * Dagger(HWP_total_operator))

#probability_HV = (Dagger(TensorProduct(H,V)) * rho_after_HWP * TensorProduct(H,V))[0]
#probability_HV.subs({theta_1:sp.pi/4, theta_2:0})

### Reduced density matrices (for either Alice or Bob)

In [4]:
def partial_trace(rho, trace_out, basis0, basis1):

    if trace_out == 0:
        operator_B0 = TensorProduct(sp.eye(2), basis0)
        operator_B1 = TensorProduct(sp.eye(2), basis1)
    elif trace_out == 1:
        operator_B0 = TensorProduct(basis0, sp.eye(2))
        operator_B1 = TensorProduct(basis1, sp.eye(2))

    return Dagger(operator_B0) * rho * operator_B0 + Dagger(operator_B1) * rho * operator_B1 

rho_Alice_after_HWP = partial_trace(rho_after_HWP, trace_out=1, basis0=H, basis1=V)
rho_Bob_after_HWP = partial_trace(rho_after_HWP, trace_out=0, basis0=H, basis1=V)

# probability alice measures H
p_H_Alice = Tr(rho_Alice_after_HWP * P_H)
p_H_Bob = Tr(rho_Bob_after_HWP * P_H)

rA = sp.simplify(p_H_Alice)
rB = sp.simplify(p_H_Bob)

print(f"Alice: {rA}")
print(f"Bob: {rB}") 

Alice: 1/2
Bob: 1/2


In [5]:
sp.simplify(partial_trace(rho_after_HWP, trace_out=1, basis0=H, basis1=V))

Matrix([
[1/2,   0],
[  0, 1/2]])

so we see that alice and bob always have a 50/50 chance of measuring H or V no matter what setting the HWP are set to

In [6]:
# chance of measuring HV or VH
Tr(rho_after_HWP * HV_operator) + Tr(rho_after_HWP*VH_operator)

sin(2*theta_1 - 2*theta_2)**2

Thus as long as
$$
\theta_1 - \theta_2 = k \frac{\pi}{4}
$$
We will only be producing HV and VH states
BUT... this is useless because we can not have the two coupled, since assuming they are infinitely far apart they can not communicate this

In [7]:
partial_trace(rho_after_HWP.subs({theta_1:0}), trace_out=1, basis0=H, basis1=V)

Matrix([
[sin(2*theta_2)**2/2 + cos(2*theta_2)**2/2,                                         0],
[                                        0, sin(2*theta_2)**2/2 + cos(2*theta_2)**2/2]])

In [8]:
# lambdify for fast eval
outcome_probabilities = sp.lambdify([theta_1, theta_2],
        sp.Matrix([
            Tr(rho_after_HWP * HH_operator),
            Tr(rho_after_HWP * HV_operator),
            Tr(rho_after_HWP * VH_operator),
            Tr(rho_after_HWP * VV_operator)
]))

Corrs = np.zeros((2,2))
for i, t1 in enumerate([0, -np.pi/8]):
    for j, t2 in enumerate([-np.pi/16, np.pi/16]):

        # N11, N10, N01, N00 
        N = np.array([0, 0, 0, 0], dtype=int)
        for _ in range(10000):
            activated_output = np.random.choice(a=[0, 1, 2, 3], p=outcome_probabilities(t1, t2)[:,0])
            N[activated_output] += 1

        Corrs[i, j] = (N[0] - N[1] - N[2] + N[3]) / N.sum()
        print(f"Corr[{i},{j}] = {Corrs[i, j]}")
        print(f"\tN[0]={N[0]}")
        print(f"\tN[1]={N[1]}")
        print(f"\tN[2]={N[2]}")
        print(f"\tN[3]={N[3]}")

S = np.abs(Corrs[0,0] + Corrs[0,1] + Corrs[1,0] - Corrs[1,1])
S

Corr[0,0] = 0.7272
	N[0]=4299
	N[1]=686
	N[2]=678
	N[3]=4337
Corr[0,1] = 0.69
	N[0]=4276
	N[1]=780
	N[2]=770
	N[3]=4174
Corr[1,0] = -0.6938
	N[0]=791
	N[1]=4248
	N[2]=4221
	N[3]=740
Corr[1,1] = 0.7232
	N[0]=4294
	N[1]=667
	N[2]=717
	N[3]=4322


2.8341999999999996

In [38]:
np.abs(1.414 - 0.70574 -0.7068 -0.70648)

0.7050200000000001

In [39]:
sp.Matrix([
            Tr(rho_after_HWP * HH_operator),
            Tr(rho_after_HWP * HV_operator),
            Tr(rho_after_HWP * VH_operator),
            Tr(rho_after_HWP * VV_operator)
]).subs({theta_1:0, theta_2:np.pi/8})

Matrix([
[0.25],
[0.25],
[0.25],
[0.25]])

Attemp to calculate S

In [40]:
Tr(rho_after_HWP * TensorProduct(P_V, P_V)).subs({theta_1:0, theta_2:0})

1/2

In [41]:
C = Tr(rho_after_HWP * HH_operator) - Tr(rho_after_HWP * HV_operator)- Tr(rho_after_HWP * VH_operator) + Tr(rho_after_HWP * VV_operator)
C = C.simplify().subs({Lambda:0*sp.pi/2})
C.simplify()


cos(4*theta_1 - 4*theta_2)

In [42]:
t11, t22 = sp.symbols("theta_11 theta_22", real=True)

S = C + C.subs({theta_2:t22}) + C.subs({theta_1:t11}) - C.subs({theta_1:t11, theta_2:t22})
S.subs({theta_1:0, theta_2:-sp.pi/16, t11:-sp.pi/8, t22:sp.pi/16})


2*sqrt(2)

QUESTIONS:

The Correlation function depends on which bell state we are receiving. Thus the ideal choice of angle settings depends on which bell state we have. 

+ Is the type 1 SPDC state we have form qutools phi+ or phi- and is it constant